In [28]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import laspy
import glob
import os.path as osp
import numpy as np


In [ ]:

class LidarToyDataset(Dataset):
    def __init__(self, files, transform=None, target_transform=None):
        self.files = files
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        filename = self.files[idx]
        cloud = laspy.read(filename)

        if self.transform:
            cloud = self.transform(cloud)
        if self.target_transform:
            labels = self.target_transform(labels)
        return cloud, labels

In [46]:
data_dir ="../data/lidar_toy/"
train_dir_query = osp.join(data_dir, "train/*.las")
train_files = glob.glob(train_dir_query)
filename = train_files[0]
las = laspy.read(filename)

In [58]:
cloud = np.asarray(
    [
        las.x - las.x.min(),
        las.y - las.y.min(),
        las.z,
        las.intensity,
        las.return_num,
        las.num_returns,
    ],
    dtype=np.float32,
)
labels = las.classification.astype(np.float32)

# get a center of a subtile
TILE_WIDTH = 1000
SUBTILE_WIDTH = 100
subtile_center_xy = (SUBTILE_WIDTH / 2) + np.random.random(size=2) * (
    TILE_WIDTH - SUBTILE_WIDTH
)
# select points of subtile
chebyshev_distance = np.max(np.abs(cloud[:2].T - subtile_center_xy), axis=1)
mask = chebyshev_distance < (SUBTILE_WIDTH / 2)
subcloud = cloud[
    :, mask
]


In [78]:
# get the mask based on the randoly chosen center of a subtile
TILE_WIDTH = 1000
SUBTILE_WIDTH = 100
subtile_center_xy = (SUBTILE_WIDTH / 2) + np.random.random(size=2) * (
    TILE_WIDTH - SUBTILE_WIDTH
)
chebyshev_distance = np.max(np.abs(cloud[:2].T - subtile_center_xy), axis=1)
mask = chebyshev_distance < (SUBTILE_WIDTH / 2)

In [76]:
np.unique(las.classification)

array([  1,   2,   6,  19,  20,  21, 104, 110, 112, 114, 115], dtype=uint8)

In [79]:
labels = las.classification.astype(np.float32)
labels = labels[mask]

In [81]:
labels.shape

(284104,)

In [37]:
las.point_format.dimensions

[DimensionInfo(name='X', kind=<DimensionKind.SignedInteger: 0>, num_bits=32, num_elements=1, is_standard=True, description='', offsets=None, scales=None),
 DimensionInfo(name='Y', kind=<DimensionKind.SignedInteger: 0>, num_bits=32, num_elements=1, is_standard=True, description='', offsets=None, scales=None),
 DimensionInfo(name='Z', kind=<DimensionKind.SignedInteger: 0>, num_bits=32, num_elements=1, is_standard=True, description='', offsets=None, scales=None),
 DimensionInfo(name='intensity', kind=<DimensionKind.UnsignedInteger: 1>, num_bits=16, num_elements=1, is_standard=True, description='', offsets=None, scales=None),
 DimensionInfo(name='return_number', kind=<DimensionKind.BitField: 3>, num_bits=4, num_elements=1, is_standard=True, description='', offsets=None, scales=None),
 DimensionInfo(name='number_of_returns', kind=<DimensionKind.BitField: 3>, num_bits=4, num_elements=1, is_standard=True, description='', offsets=None, scales=None),
 DimensionInfo(name='synthetic', kind=<Dimen